In [3]:
import boto3
import urllib3
import boto3
import pandas as pd
import io
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

%update_token

Token successfully refreshed.


In [4]:
# Retrieve the list of existing buckets
#os.environ["AWS_ENDPOINT_URL"] = "http://external-s3-service.ezdata-system.svc.cluster.local:30000"
os.environ["AWS_ENDPOINT_URL"] = 'http://harshaledf-service.ezdata-system.svc.cluster.local:30000'
os.environ['AWS_ACCESS_KEY_ID'] = os.environ['AUTH_TOKEN']
os.environ['AWS_SECRET_ACCESS_KEY'] = 's3'
s3 = boto3.client("s3", endpoint_url=os.environ["AWS_ENDPOINT_URL"] ,
                  aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
                  aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'])
response = s3.list_buckets()
# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  checkpoints
  experiments
  icebergdata
  processed1
  secondbucket
  tenanta


In [5]:
# Code to copy finacial.csv from ezua-tutorials to ezaf-demo bucket at data/ for Financial Time Series Example

s3 = boto3.client("s3", verify=False)
bucket = 'ezaf-demo'
source_file = '/mnt/shared/harshal/financial.csv'
dest_object = 'data/financial.csv'
# Check whether bucket is already created
buckets = s3.list_buckets()
bucket_exists = False
available_buckets = buckets["Buckets"]
for available_bucket in available_buckets:
    if available_bucket["Name"] == bucket:
        bucket_exists = True
        break
# Create bucket if not exists
if not bucket_exists:
    s3.create_bucket(Bucket=bucket)
# Upload file
s3.upload_file(Filename=source_file, Bucket=bucket, Key=dest_object) 

Found credentials in environment variables.
Found endpoint for s3 via: environment_global.


In [6]:
xobject = s3.get_object(Bucket='ezaf-demo',Key='data/financial.csv')
df = pd.read_csv(io.StringIO(xobject["Body"].read().decode('utf-8')))

In [7]:
df.head()

,snp_close,nyse_close,djia_close,nikkei_close,hangseng_close,ftse_close,dax_close,aord_close
0,2098.120117,11112.169922,18067.230469,19623.839844,27406.179688,6932.799805,11471.410156,5672.100098
1,1413.199951,8233.910156,13092.160156,9050.219727,22110.330078,5867.600098,7362.850098,4482.299805
2,1856.619995,10433.870117,16322.059570,14695.030273,22064.529297,6614.600098,9586.190430,5375.799805
3,1140.199951,7280.069824,10787.049805,9368.349609,22357.169922,5547.600098,6228.020020,4635.899902
4,2106.959961,11143.450195,18037.269531,20132.900391,27932.849609,7027.200195,11866.370117,5811.799805


In [9]:
#add S3 secret for Spark 
import kfp
kfp_client = kfp.Client()
namespace = kfp_client.get_user_namespace()
!sed -e "s/\$AUTH_TOKEN/$AUTH_TOKEN/" /mnt/shared/harshal/object_store_secret.yaml.tpl > /mnt/shared/harshal/object_store_secret.yaml


In [11]:
!kubectl apply -f /mnt/shared/harshal/object_store_secret.yaml

secret/spark-s3-creds created
